In [44]:
import pandas as pd
import numpy as np
from operator import itemgetter 

## Identify Overlapping Peaks of Two Peak Tables

In [2]:
def ppm_calculation(x, y):
    return abs(x-y)*10e6/y

In [3]:
def nearest_value(lst, K):
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

In [79]:
def MS1_match(mz, rt, mz_list, rt_list, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
#    print("enter match")
    df = pd.DataFrame(list(zip(mz_list, rt_list)),
               columns =[col_name_mz, col_name_rt])
    if len(df[(ppm_calculation(mz, df[col_name_mz])<=ppm_threshold) & (abs(df[col_name_rt] - rt) < rt_tolerance)])>0:
        return True
    else:
        return False

In [129]:
def overlapping_peaks(t_1, t_2, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
    flag = 0
    flagged_rows = []
    total = t_2.shape[0]
    if type(t_1[col_name_mz].iloc[0]) == np.float64:
        no_string = 1
    else:
        no_string = 0      
    mz_list = []
    rt_list = []
    if no_string == 1:
        mz_list = list(t_1[col_name_mz])
        rt_list = list(t_1[col_name_rt])
    else:
        for mz_element in t_1[col_name_mz].str.split("_"):
            mz_list += [float(x) for x in mz_element]
        for rt_element in t_1[col_name_rt].str.split("_"):
            rt_list += [float(x) for x in rt_element]
    for i in range(total):
        print(str(i*100/total) + "%", end="\r")
        second_flag = 0
        if (type(t_2[col_name_mz].iloc[i]) == np.float64) or ("_" not in t_2[col_name_mz].iloc[i]):
            mz = float(t_2[col_name_mz].iloc[i])
            rt = float(t_2[col_name_rt].iloc[i])
            if MS1_match(mz, rt, mz_list, rt_list, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
                continue
            else:
                flag = 1
        else:
            for j in range(len(t_2[col_name_mz].iloc[i].split("_"))):
                mz = float(t_2[col_name_mz].iloc[i].split("_")[j])
                rt = float(t_2[col_name_rt].iloc[i].split("_")[j])
                nearest_id = mz_list.index(nearest_value(mz_list, mz))
                if MS1_match(mz, rt, mz_list, rt_list, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
                    second_flag = 1
                    continue
            if second_flag == 0:
                flag = 1
        if flag == 1:
            flagged_rows.append(i)
    return t_2.drop(labels=flagged_rows, axis=0)

In [54]:
def quantification(t_1, t_2, t_merged, col_name_annotation_status, desired_identification_status, col_name_metabolite_name):
    n_rows_t_1 = t_1.shape[0]
    n_rows_t_2 = t_2.shape[0]
    n_rows_t_merged = t_merged.shape[0]
    
    n_annotated_rows_t_1 = t_1[t_1[col_name_annotation_status] == desired_identification_status].shape[0]
    n_annotated_rows_t_2 = t_2[t_2[col_name_annotation_status] == desired_identification_status].shape[0]
    n_annotated_rows_t_merged = t_merged[t_merged[col_name_annotation_status] == desired_identification_status].shape[0]
    
    tpr_t_1 = round(n_annotated_rows_t_1/n_rows_t_1, 4)
    tpr_t_2 = round(n_annotated_rows_t_2/n_rows_t_2, 4)
    tpr_t_merged = round(n_annotated_rows_t_merged/n_rows_t_merged, 4)
    
    n_deduplicated_metabolites_t_1 = t_1[t_1[col_name_annotation_status] == desired_identification_status].drop_duplicates(subset = col_name_metabolite_name).shape[0]
    n_deduplicated_metabolites_t_2 = t_2[t_2[col_name_annotation_status] == desired_identification_status].drop_duplicates(subset = col_name_metabolite_name).shape[0]
    n_deduplicated_metabolites_t_merged = t_merged[t_merged[col_name_annotation_status] == desired_identification_status].drop_duplicates(subset = col_name_metabolite_name).shape[0]
    
    d = {
        'Peak Tables': ['Peak Table 1', 'Peak Table 2', 'Merged Peak Table'],
        'Total Number of Peaks': [n_rows_t_1, n_rows_t_2, n_rows_t_merged],
        'Number of True Peaks': [n_annotated_rows_t_1, n_annotated_rows_t_2, n_annotated_rows_t_merged],
        'True Positive Rate': [tpr_t_1, tpr_t_2, tpr_t_merged],
        'Number of Distinct Metabolites': [n_deduplicated_metabolites_t_1, n_deduplicated_metabolites_t_2, n_deduplicated_metabolites_t_merged]
    }
    
    df = pd.DataFrame(data=d)
    
    return df

### Local CMD VS HPC CMD

#### MS-DIAL

In [84]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-dial/AlignResult-2023282236.txt", delimiter="\t", header=4)

In [85]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-dial/AlignResult-2023291844.txt", delimiter="\t", header=4)

In [86]:
len(table_1)

25952

In [88]:
len(table_2)

25447

In [89]:
table_merged_msdial_lchc = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [90]:
len(table_merged_msdial_lchc)

18856

#### MS-FLO

In [91]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-flo/AlignResult-2023282236_processed.txt", delimiter="\t")

In [92]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-flo/AlignResult-2023291844_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [93]:
table_merged_msflo_lchc = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

mz value is a string%
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string


In [94]:
len(table_merged_msflo_lchc)

18807

### HPC NF VS Local NF

#### MS-DIAL

In [95]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-dial/AlignResult-2023281418.txt", delimiter="\t", header=4)

In [96]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-dial/AlignResult-20232101814.txt", delimiter="\t", header=4)

In [97]:
len(table_1)

25952

In [98]:
len(table_2)

25447

In [99]:
table_merged_msdial_hnln = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [100]:
len(table_merged_msdial_hnln)

18856

#### MS-FLO

In [101]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-flo/AlignResult-2023281418_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [102]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-flo/AlignResult-20232101814_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [103]:
len(table_1)

25678

In [104]:
len(table_2)

25414

In [105]:
table_merged_msflo_hnln = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

mz value is a string%
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string


In [106]:
len(table_merged_msflo_hnln)

18807

### HPC CMD VS HPC NF

#### MS-DIAL

In [107]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-dial/AlignResult-2023282236.txt", delimiter="\t", header=4)

In [108]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-dial/AlignResult-2023281418.txt", delimiter="\t", header=4)

In [109]:
len(table_1)

25952

In [110]:
len(table_2)

25952

In [111]:
peak_merged_msdial_hchn = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [112]:
len(peak_merged_msdial_hchn)

25952

#### MS-FLO

In [113]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_cmd/ms-flo/AlignResult-2023282236_processed.txt", delimiter="\t")

In [114]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/hpc_nf/ms-flo/AlignResult-2023281418_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [115]:
len(table_1)

25678

In [116]:
len(table_2)

25678

In [117]:
peak_merged_msflo_hchn = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

mz value is a string%
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is 

In [118]:
len(peak_merged_msflo_hchn)

25678

### Local CMD VS Local NF

#### MS-DIAL

In [119]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-dial/AlignResult-2023291844.txt", delimiter="\t", header=4)

In [120]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-dial/AlignResult-20232101814.txt", delimiter="\t", header=4)

In [121]:
len(table_1)

25447

In [122]:
len(table_2)

25447

In [123]:
peak_merged_msdial_lcln = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

In [124]:
len(peak_merged_msdial_lcln)

25447

#### MS-FLO

In [130]:
table_1 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_cmd/ms-flo/AlignResult-2023291844_processed.txt", delimiter="\t")

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [131]:
table_2 = pd.read_csv("../../nextflow4metabolomics_manuscripts/nextflow4ms-dial/peak_table/local_nf/ms-flo/AlignResult-20232101814_processed.txt", delimiter="\t")

In [132]:
len(table_1)

25414

In [133]:
len(table_2)

25414

In [127]:
peak_merged_msflo_lcln = overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

mz value is a string%
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string
mz value is a string


In [128]:
len(peak_merged_msflo_lcln)

25414

## Other Tests

In [175]:
table_1[table_1["Annotation tag (VS1.0)"] == "annotated by user-defined text library"].drop_duplicates(subset = "Metabolite name").shape

(660, 42)

In [173]:
table_1[table_1["Annotation tag (VS1.0)"] == "annotated by user-defined text library"]

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,SA1,SA2,SA3,SA4,SA5,SB1,SB2,SB3,SB4,SB5
263,263,0.593,109.07635,C6H8N2,[M+H]+,NaN,1.0,False,0.59,109.07630,...,4.606381e+07,52049700.00,48648996.00,48866940.00,50095360.00,49510468.00,4.723662e+07,4.427318e+07,48335940.00,48993324.00
265,265,0.528,109.07640,C6H8N2,[M+H]+,NaN,1.0,False,0.59,109.07630,...,2.600366e+07,25304870.00,26239746.00,25760260.00,26979930.00,27128062.00,2.614744e+07,2.504948e+07,26958778.00,26436970.00
449,449,0.581,114.12804,C7H15N,[M+H]+,NaN,1.0,False,0.58,114.12800,...,5.413586e+07,47681840.00,45654152.00,48545060.00,44667432.00,48684136.00,4.539799e+07,4.475062e+07,44759100.00,48471236.00
562,562,5.085,119.06066,C7H6N2,[M+H]+,NaN,1.0,False,5.08,119.06060,...,5.412722e+07,58578392.00,48993616.00,51911076.00,49807312.00,47893944.00,4.601670e+07,4.810860e+07,49952244.00,49673528.00
740,740,0.672,126.05515,C6H7NO2,[M+H]+,NaN,0.8,False,0.63,126.05510,...,1.217748e+07,10208069.00,10337025.00,10018535.00,9628490.00,8517147.00,9.200289e+06,9.645849e+06,9413457.00,10822839.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25021,25021,31.697,980.57556,C53H83NO14,[M+Na]+,adduct linked to 24934_[M+H]+,1.0,False,31.70,980.57220,...,4.133900e+07,47972908.00,49342812.00,47552156.00,48896308.00,50457108.00,4.722142e+07,4.351532e+07,44695528.00,45775832.00
25142,25142,4.495,1019.45032,C49H66N10O10S2,[M+H]+,ion correlated with 2281,0.5,False,4.45,1019.45007,...,2.092166e+06,1821976.80,1716858.20,1969067.80,1968747.90,791041.56,1.685933e+05,5.777482e+04,78292.47,114933.63
25143,25143,4.390,1019.45239,C49H66N10O10S2,[M+H]+,ion correlated with 1163,0.3,False,4.45,1019.45007,...,4.415434e+04,107192.33,141995.03,103186.31,101158.01,371036.10,2.422940e+05,1.903450e+06,2249673.50,2219476.80
25144,25144,4.453,1019.45251,C49H66N10O10S2,[M+H]+,NaN,0.3,False,4.45,1019.45007,...,4.510274e+05,1864471.90,660375.30,924083.56,1182922.50,2023898.00,2.069770e+06,7.085063e+05,434584.72,76172.26


In [168]:
table_merged.shape

(17245, 47)

In [178]:
summary_df = quantification(table_1, table_2, table_merged, "Annotation tag (VS1.0)", "annotated by user-defined text library", "Metabolite name")

In [179]:
summary_df

,Peak Tables,Total Number of Peaks,Number of True Peaks,True Positive Rate,Number of Distinct Metabolites
0,Peak Table 1,25447,1031,0.0405,660
1,Peak Table 2,25447,1031,0.0405,660
2,Merged Peak Table,17245,924,0.0536,629
